In [1]:
import ast
import pandas as pd

# ------------------------------------------------------------------
# Config
# ------------------------------------------------------------------
INPUT_CSV = "pde_sum_all_files.csv"
OUTPUT_CSV = "pde_sum_all_files_with_names_newlight.csv"

# ------------------------------------------------------------------
# Mapping: (ADC, local channels) -> trap/tile name
# Names built from "Light Tile Type" + "Light Tile #"
# ------------------------------------------------------------------
# TRAP_NAME_MAP = [
#     # ----- ADC 0 (ACLs module 0, made-up names) -----
#     dict(adc=0, channels=[4,5,6,7,8,9], name="ACL 2.01"),               # -z, bottom, tpc 0
#     dict(adc=0, channels=[10,11,12,13,14,15], name="ACL 2.02"),         # -z, top, tpc 0
#     dict(adc=0, channels=[20,21,22,23,24,25], name="ACL 2.03"),         # +z bottom, tpc 0
#     dict(adc=0, channels=[26,27,28,29,30,31], name="ACL 2.04"),         # +z top, tpc 0
#     dict(adc=0, channels=[52,53,54,55,56,57], name="ACL 2.05"),         # -z bottom, tpc 1
#     dict(adc=0, channels=[58,59,60,61,62,63], name="ACL 2.06"),         # -z top, tpc 1
#     dict(adc=0, channels=[36,37,38,39,40,41], name="ACL 2.07"),         # +z bottom, tpc 1
#     dict(adc=0, channels=[42,43,44,45,46,47], name="ACL 2.08"),         # +z top, tpc 1
    
#     # ----- ADC 1 (LCMs module 0, made-up names) -----
#     dict(adc=1, channels=[4,5,6,7,8,9], name="LCM 100,101,102"),        # -z, bottom, tpc 0
#     dict(adc=1, channels=[10,11,12,13,14,15], name="LCM 103,104,105"),  # -z, top, tpc 0
#     dict(adc=1, channels=[20,21,22,23,24,25], name="LCM 106,107,108"),  # +z bottom, tpc 0
#     dict(adc=1, channels=[26,27,28,29,30,31], name="LCM 109,110,111"),  # +z top, tpc 0
#     dict(adc=1, channels=[52,53,54,55,56,57], name="LCM 112,113,114"),  # -z bottom, tpc 1
#     dict(adc=1, channels=[58,59,60,61,62,63], name="LCM 115,116,117"),  # -z top, tpc 1
#     dict(adc=1, channels=[36,37,38,39,40,41], name="LCM 118,119,120"),  # +z bottom, tpc 1
#     dict(adc=1, channels=[42,43,44,45,46,47], name="LCM 121,122,123"),  # +z top, tpc 1
    
#     # ----- ADC 2 (ACLs module 1, made-up names) -----
#     dict(adc=2, channels=[4,5,6,7,8,9], name="ACL 2.09"),               # -z, bottom, tpc 2
#     dict(adc=2, channels=[10,11,12,13,14,15], name="ACL 2.10"),         # -z, top, tpc 2
#     dict(adc=2, channels=[20,21,22,23,24,25], name="ACL 2.11"),         # +z bottom, tpc 2
#     dict(adc=2, channels=[26,27,28,29,30,31], name="ACL 2.12"),         # +z top, tpc 2
#     dict(adc=2, channels=[52,53,54,55,56,57], name="ACL 2.13"),         # -z bottom, tpc 3
#     dict(adc=2, channels=[58,59,60,61,62,63], name="ACL 2.14"),         # -z top, tpc 3
#     dict(adc=2, channels=[36,37,38,39,40,41], name="ACL 2.15"),         # +z bottom, tpc 3
#     dict(adc=2, channels=[42,43,44,45,46,47], name="ACL 2.16"),         # +z top, tpc 3

#     # ----- ADC 3 (LCMs module 1, made-up names) -----
#     dict(adc=3, channels=[20,21,22,23,24,25], name="LCM 124,125,126"),  # -z, bottom, tpc 2
#     dict(adc=3, channels=[26,27,28,29,30,31], name="LCM 127,128,129"),  # -z, top, tpc 2
#     dict(adc=3, channels=[4,5,6,7,8,9], name="LCM 130,131,132"),        # +z bottom, tpc 2
#     dict(adc=3, channels=[10,11,12,13,14,15], name="LCM 133,134,135"),  # +z top, tpc 2
#     dict(adc=3, channels=[52,53,54,55,56,57], name="LCM 136,137,138"),  # -z bottom, tpc 3
#     dict(adc=3, channels=[58,59,60,61,62,63], name="LCM 139,140,141"),  # -z top, tpc 3
#     dict(adc=3, channels=[36,37,38,39,40,41], name="LCM 142,143,144"),  # +z bottom, tpc 3
#     dict(adc=3, channels=[42,43,44,45,46,47], name="LCM 145,146,147"),  # +z top, tpc 3

#     # ----- ADC 4 (ACLs module 2, made-up names) -----
#     dict(adc=4, channels=[42,43,44,45,46,47], name="ACL 2.17"),         # -z, bottom, tpc 4
#     dict(adc=4, channels=[36,37,38,39,40,41], name="ACL 2.18"),         # -z, top, tpc 4
#     dict(adc=4, channels=[52,53,54,55,56,57], name="LCM 148,149,150"),  # -z top, tpc 4
#     dict(adc=4, channels=[58,59,60,61,62,63], name="LCM 151,152,153"),  # +z bottom, tpc 4
#     dict(adc=4, channels=[26,27,28,29,30,31], name="ACL 2.19"),         # -z bottom, tpc 5
#     dict(adc=4, channels=[20,21,22,23,24,25], name="ACL 2.20"),         # -z top, tpc 5
#     dict(adc=4, channels=[10,11,12,13,14,15], name="ACL 2.21"),         # +z bottom, tpc 5
#     dict(adc=4, channels=[4,5,6,7,8,9], name="ACL 2.22"),               # +z top, tpc 5

#     # ----- ADC 5 (LCMs module 2, made-up names) -----
#     dict(adc=5, channels=[42,43,44,45,46,47], name="LCM 154,155,156"),  # -z, bottom, tpc 4
#     dict(adc=5, channels=[58,59,60,61,62,63], name="ACL 2.23"),         # +z, bottom, tpc 4
#     dict(adc=5, channels=[52,53,54,55,56,57], name="ACL 2.24"),         # +z top, tpc 4
#     dict(adc=5, channels=[36,37,38,39,40,41], name="LCM 157,158,159"),  # +z top, tpc 4
#     dict(adc=5, channels=[26,27,28,29,30,31], name="LCM 160,161,162"),  # -z bottom, tpc 5
#     dict(adc=5, channels=[20,21,22,23,24,25], name="LCM 163,164,165"),  # -z top, tpc 5
#     dict(adc=5, channels=[10,11,12,13,14,15], name="LCM 166,167,168"),  # +z bottom, tpc 5
#     dict(adc=5, channels=[4,5,6,7,8,9], name="LCM 169,170,171"),        # +z top, tpc 5

#     # ----- ADC 6 (ACLs module 3, made-up names) -----
#     dict(adc=6, channels=[4,5,6,7,8,9], name="ACL 2.25"),               # -z, bottom, tpc 6
#     dict(adc=6, channels=[10,11,12,13,14,15], name="ACL 2.26"),         # -z, top, tpc 6
#     dict(adc=6, channels=[20,21,22,23,24,25], name="ACL 2.27"),         # +z bottom, tpc 6
#     dict(adc=6, channels=[26,27,28,29,30,31], name="ACL 2.28"),         # +z top, tpc 6
#     dict(adc=6, channels=[52,53,54,55,56,57], name="ACL 2.29"),         # -z bottom, tpc 7
#     dict(adc=6, channels=[58,59,60,61,62,63], name="ACL 2.30"),         # -z top, tpc 7
#     dict(adc=6, channels=[36,37,38,39,40,41], name="ACL 2.31"),         # +z bottom, tpc 7
#     dict(adc=6, channels=[42,43,44,45,46,47], name="ACL 2.32"),         # +z top, tpc 7

#     # ----- ADC 7 (LCMs module 3, made-up names) -----
#     dict(adc=7, channels=[4,5,6,7,8,9], name="LCM 172,173,174"),        # -z, bottom, tpc 6
#     dict(adc=7, channels=[10,11,12,13,14,15], name="LCM 175,176,177"),  # -z, top, tpc 6
#     dict(adc=7, channels=[20,21,22,23,24,25], name="LCM 178,179,180"),  # +z bottom, tpc 6
#     dict(adc=7, channels=[26,27,28,29,30,31], name="LCM 181,182,183"),  # +z top, tpc 6
#     dict(adc=7, channels=[52,53,54,55,56,57], name="LCM 184,185,186"),  # -z bottom, tpc 7
#     dict(adc=7, channels=[58,59,60,61,62,63], name="LCM 187,188,189"),  # -z top, tpc 7
#     dict(adc=7, channels=[36,37,38,39,40,41], name="LCM 190,191,192"),  # +z bottom, tpc 7
#     dict(adc=7, channels=[42,43,44,45,46,47], name="LCM 193,194,195"),  # +z top, tpc 7
# ]

TRAP_NAME_MAP = [

    # ======================
    # TPC 0
    # ======================
    dict(tpc=0, trap=0,  channels=[4,5,6,7,8,9],           name="ACL 2.01"),
    dict(tpc=0, trap=1,  channels=[4,5],                   name="LCM 100"),
    dict(tpc=0, trap=2,  channels=[6,7],                   name="LCM 101"),
    dict(tpc=0, trap=3,  channels=[8,9],                   name="LCM 102"),
    dict(tpc=0, trap=4,  channels=[10,11,12,13,14,15],     name="ACL 2.02"),
    dict(tpc=0, trap=5,  channels=[10,11],                 name="LCM 103"),
    dict(tpc=0, trap=6,  channels=[12,13],                 name="LCM 104"),
    dict(tpc=0, trap=7,  channels=[14,15],                 name="LCM 105"),
    dict(tpc=0, trap=8,  channels=[20,21,22,23,24,25],     name="ACL 2.03"),
    dict(tpc=0, trap=9,  channels=[20,21],                 name="LCM 106"),
    dict(tpc=0, trap=10, channels=[22,23],                 name="LCM 107"),
    dict(tpc=0, trap=11, channels=[24,25],                 name="LCM 108"),
    dict(tpc=0, trap=12, channels=[26,27,28,29,30,31],     name="ACL 2.04"),
    dict(tpc=0, trap=13, channels=[26,27],                 name="LCM 109"),
    dict(tpc=0, trap=14, channels=[28,29],                 name="LCM 110"),
    dict(tpc=0, trap=15, channels=[30,31],                 name="LCM 111"),

    # ======================
    # TPC 1
    # ======================
    dict(tpc=1, trap=0,  channels=[52,53,54,55,56,57],     name="ACL 2.05"),
    dict(tpc=1, trap=1,  channels=[52,53],                 name="LCM 112"),
    dict(tpc=1, trap=2,  channels=[54,55],                 name="LCM 113"),
    dict(tpc=1, trap=3,  channels=[56,57],                 name="LCM 114"),
    dict(tpc=1, trap=4,  channels=[58,59,60,61,62,63],     name="ACL 2.06"),
    dict(tpc=1, trap=5,  channels=[58,59],                 name="LCM 115"),
    dict(tpc=1, trap=6,  channels=[60,61],                 name="LCM 116"),
    dict(tpc=1, trap=7,  channels=[62,63],                 name="LCM 117"),
    dict(tpc=1, trap=8,  channels=[36,37,38,39,40,41],     name="ACL 2.07"),
    dict(tpc=1, trap=9,  channels=[36,37],                 name="LCM 118"),
    dict(tpc=1, trap=10, channels=[38,39],                 name="LCM 119"),
    dict(tpc=1, trap=11, channels=[40,41],                 name="LCM 120"),
    dict(tpc=1, trap=12, channels=[42,43,44,45,46,47],     name="ACL 2.08"),
    dict(tpc=1, trap=13, channels=[42,43],                 name="LCM 121"),
    dict(tpc=1, trap=14, channels=[44,45],                 name="LCM 122"),
    dict(tpc=1, trap=15, channels=[46,47],                 name="LCM 123"),

    # ======================
    # TPC 2
    # ======================
    dict(tpc=2, trap=0,  channels=[4,5,6,7,8,9],           name="ACL 2.09"),
    dict(tpc=2, trap=1,  channels=[20,21],                 name="LCM 124"),
    dict(tpc=2, trap=2,  channels=[22,23],                 name="LCM 125"),
    dict(tpc=2, trap=3,  channels=[24,25],                 name="LCM 126"),
    dict(tpc=2, trap=4,  channels=[10,11,12,13,14,15],     name="ACL 2.10"),
    dict(tpc=2, trap=5,  channels=[26,27],                 name="LCM 127"),
    dict(tpc=2, trap=6,  channels=[28,29],                 name="LCM 128"),
    dict(tpc=2, trap=7,  channels=[30,31],                 name="LCM 129"),
    dict(tpc=2, trap=8,  channels=[20,21,22,23,24,25],     name="ACL 2.11"),
    dict(tpc=2, trap=9,  channels=[4,5],                   name="LCM 130"),
    dict(tpc=2, trap=10, channels=[6,7],                   name="LCM 131"),
    dict(tpc=2, trap=11, channels=[8,9],                   name="LCM 132"),
    dict(tpc=2, trap=12, channels=[26,27,28,29,30,31],     name="ACL 2.12"),
    dict(tpc=2, trap=13, channels=[10,11],                 name="LCM 133"),
    dict(tpc=2, trap=14, channels=[12,13],                 name="LCM 134"),
    dict(tpc=2, trap=15, channels=[14,15],                 name="LCM 135"),

    # ======================
    # TPC 3
    # ======================
    dict(tpc=3, trap=0,  channels=[52,53,54,55,56,57],     name="ACL 2.13"),
    dict(tpc=3, trap=1,  channels=[52,53],                 name="LCM 136"),
    dict(tpc=3, trap=2,  channels=[54,55],                 name="LCM 137"),
    dict(tpc=3, trap=3,  channels=[56,57],                 name="LCM 138"),
    dict(tpc=3, trap=4,  channels=[58,59,60,61,62,63],     name="ACL 2.14"),
    dict(tpc=3, trap=5,  channels=[58,59],                 name="LCM 139"),
    dict(tpc=3, trap=6,  channels=[60,61],                 name="LCM 140"),
    dict(tpc=3, trap=7,  channels=[62,63],                 name="LCM 141"),
    dict(tpc=3, trap=8,  channels=[36,37,38,39,40,41],     name="ACL 2.15"),
    dict(tpc=3, trap=9,  channels=[36,37],                 name="LCM 142"),
    dict(tpc=3, trap=10, channels=[38,39],                 name="LCM 143"),
    dict(tpc=3, trap=11, channels=[40,41],                 name="LCM 144"),
    dict(tpc=3, trap=12, channels=[42,43,44,45,46,47],     name="ACL 2.16"),
    dict(tpc=3, trap=13, channels=[42,43],                 name="LCM 145"),
    dict(tpc=3, trap=14, channels=[44,45],                 name="LCM 146"),
    dict(tpc=3, trap=15, channels=[46,47],                 name="LCM 147"),

    # ======================
    # TPC 4
    # ======================
    dict(tpc=4, trap=0,  channels=[42,43,44,45,46,47],     name="ACL 2.17"),
    dict(tpc=4, trap=1,  channels=[42,43],                 name="LCM 148"),
    dict(tpc=4, trap=2,  channels=[44,45],                 name="LCM 149"),
    dict(tpc=4, trap=3,  channels=[46,47],                 name="LCM 150"),
    dict(tpc=4, trap=4,  channels=[36,37,38,39,40,41],     name="ACL 2.18"),
    dict(tpc=4, trap=5,  channels=[52,53],                 name="LCM 151"),
    dict(tpc=4, trap=6,  channels=[54,55],                 name="LCM 152"),
    dict(tpc=4, trap=7,  channels=[56,57],                 name="LCM 153"),
    dict(tpc=4, trap=8,  channels=[58,59,60,61,62,63],     name="ACL 2.19"),
    dict(tpc=4, trap=9,  channels=[58,59],                 name="LCM 154"),
    dict(tpc=4, trap=10, channels=[60,61],                 name="LCM 155"),
    dict(tpc=4, trap=11, channels=[62,63],                 name="LCM 156"),
    dict(tpc=4, trap=12, channels=[52,53,54,55,56,57],     name="ACL 2.20"),
    dict(tpc=4, trap=13, channels=[36,37],                 name="LCM 157"),
    dict(tpc=4, trap=14, channels=[38,39],                 name="LCM 158"),
    dict(tpc=4, trap=15, channels=[40,41],                 name="LCM 159"),

    # ======================
    # TPC 5
    # ======================
    dict(tpc=5, trap=0,  channels=[26,27,28,29,30,31],     name="ACL 2.21"),
    dict(tpc=5, trap=1,  channels=[26,27],                 name="LCM 160"),
    dict(tpc=5, trap=2,  channels=[28,29],                 name="LCM 161"),
    dict(tpc=5, trap=3,  channels=[30,31],                 name="LCM 162"),
    dict(tpc=5, trap=4,  channels=[20,21,22,23,24,25],     name="ACL 2.22"),
    dict(tpc=5, trap=5,  channels=[20,21],                 name="LCM 163"),
    dict(tpc=5, trap=6,  channels=[22,23],                 name="LCM 164"),
    dict(tpc=5, trap=7,  channels=[24,25],                 name="LCM 165"),
    dict(tpc=5, trap=8,  channels=[10,11,12,13,14,15],     name="ACL 2.23"),
    dict(tpc=5, trap=9,  channels=[10,11],                 name="LCM 166"),
    dict(tpc=5, trap=10, channels=[12,13],                 name="LCM 167"),
    dict(tpc=5, trap=11, channels=[14,15],                 name="LCM 168"),
    dict(tpc=5, trap=12, channels=[4,5,6,7,8,9],           name="ACL 2.24"),
    dict(tpc=5, trap=13, channels=[4,5],                   name="LCM 169"),
    dict(tpc=5, trap=14, channels=[6,7],                   name="LCM 170"),
    dict(tpc=5, trap=15, channels=[8,9],                   name="LCM 171"),

    # ======================
    # TPC 6
    # ======================
    dict(tpc=6, trap=0,  channels=[4,5,6,7,8,9],           name="ACL 2.25"),
    dict(tpc=6, trap=1,  channels=[4,5],                   name="LCM 172"),
    dict(tpc=6, trap=2,  channels=[6,7],                   name="LCM 173"),
    dict(tpc=6, trap=3,  channels=[8,9],                   name="LCM 174"),
    dict(tpc=6, trap=4,  channels=[10,11,12,13,14,15],     name="ACL 2.26"),
    dict(tpc=6, trap=5,  channels=[10,11],                 name="LCM 175"),
    dict(tpc=6, trap=6,  channels=[12,13],                 name="LCM 176"),
    dict(tpc=6, trap=7,  channels=[14,15],                 name="LCM 177"),
    dict(tpc=6, trap=8,  channels=[20,21,22,23,24,25],     name="ACL 2.27"),
    dict(tpc=6, trap=9,  channels=[20,21],                 name="LCM 178"),
    dict(tpc=6, trap=10, channels=[22,23],                 name="LCM 179"),
    dict(tpc=6, trap=11, channels=[24,25],                 name="LCM 180"),
    dict(tpc=6, trap=12, channels=[26,27,28,29,30,31],     name="ACL 2.28"),
    dict(tpc=6, trap=13, channels=[26,27],                 name="LCM 181"),
    dict(tpc=6, trap=14, channels=[28,29],                 name="LCM 182"),
    dict(tpc=6, trap=15, channels=[30,31],                 name="LCM 183"),

    # ======================
    # TPC 7
    # ======================
    dict(tpc=7, trap=0,  channels=[52,53,54,55,56,57],     name="ACL 2.29"),
    dict(tpc=7, trap=1,  channels=[52,53],                 name="LCM 184"),
    dict(tpc=7, trap=2,  channels=[54,55],                 name="LCM 185"),
    dict(tpc=7, trap=3,  channels=[56,57],                 name="LCM 186"),
    dict(tpc=7, trap=4,  channels=[58,59,60,61,62,63],     name="ACL 2.30"),
    dict(tpc=7, trap=5,  channels=[58,59],                 name="LCM 187"),
    dict(tpc=7, trap=6,  channels=[60,61],                 name="LCM 188"),
    dict(tpc=7, trap=7,  channels=[62,63],                 name="LCM 189"),
    dict(tpc=7, trap=8,  channels=[36,37,38,39,40,41],     name="ACL 2.31"),
    dict(tpc=7, trap=9,  channels=[36,37],                 name="LCM 190"),
    dict(tpc=7, trap=10, channels=[38,39],                 name="LCM 191"),
    dict(tpc=7, trap=11, channels=[40,41],                 name="LCM 192"),
    dict(tpc=7, trap=12, channels=[42,43,44,45,46,47],     name="ACL 2.32"),
    dict(tpc=7, trap=13, channels=[42,43],                 name="LCM 193"),
    dict(tpc=7, trap=14, channels=[44,45],                 name="LCM 194"),
    dict(tpc=7, trap=15, channels=[46,47],                 name="LCM 195"),
]

# Normalize channels to sorted tuples for safe matching
TRAP_NAME_LOOKUP = {
    (entry["tpc"], tuple(sorted(entry["channels"]))): entry["name"]
    for entry in TRAP_NAME_MAP
}


# ------------------------------------------------------------------
# Helpers
# ------------------------------------------------------------------
def parse_ch_list(value):
    """
    Turn the ch_list column into a Python list of ints.
    Handles things like:
      "[0, 4]"       -> [0, 4]
      "0,4"          -> [0, 4]
      already-a-list -> itself
    """
    if isinstance(value, list):
        return value

    if pd.isna(value):
        return []

    if isinstance(value, str):
        text = value.strip()
        # case: string representation of a Python list
        if text.startswith("[") and text.endswith("]"):
            try:
                parsed = ast.literal_eval(text)
                if isinstance(parsed, (list, tuple)):
                    return [int(x) for x in parsed]
            except Exception:
                pass

        # fallback: comma-separated
        parts = [p.strip() for p in text.split(",") if p.strip()]
        out = []
        for p in parts:
            try:
                out.append(int(p))
            except ValueError:
                # ignore non-numeric garbage if any
                pass
        return out

    # anything else -> empty
    return []


def find_trap_name(tpc, trap, ch_list):
    """
    Find trap name by:
      - matching tpc, trap, channel list
    If nothing matches, return "".
    """
    key = (int(tpc), tuple(sorted(ch_list)))

    try:
        return TRAP_NAME_LOOKUP[key]
    except KeyError:
        return "NoName"


# ------------------------------------------------------------------
# Main
# ------------------------------------------------------------------
def main():
    # Read original CSV
    df = pd.read_csv(INPUT_CSV)
    # Parse ch_list into actual lists of ints
    df["ch_list_parsed"] = df["ch_list"].apply(parse_ch_list)
    print(df.head())
    # Compute trap_name for each row
    df["trap_name"] = [
        find_trap_name(row["tpc"], row["trap"], row["ch_list_parsed"])
        for _, row in df.iterrows()
    ]

    # Drop helper column
    df = df.drop(columns=["ch_list_parsed"])

    # Write new CSV with extra column
    df.to_csv(OUTPUT_CSV, index=False)


if __name__ == "__main__":
    main()


   global_trap_id  tpc  trap                   ch_list     PE_meas  \
0               0    0     0        [4, 5, 6, 7, 8, 9]    0.000000   
1               1    0     1                    [4, 5]  206.449211   
2               2    0     2                    [6, 7]  378.757710   
3               3    0     3                    [8, 9]  730.400238   
4               4    0     4  [10, 11, 12, 13, 14, 15]  388.724642   

   PE_meas_noLTcr  PE_exp  PE_exp_noLTcr            ch_list_parsed  
0             0.0     0.0            0.0        [4, 5, 6, 7, 8, 9]  
1             0.0     0.0            0.0                    [4, 5]  
2             0.0     0.0            0.0                    [6, 7]  
3             0.0     0.0            0.0                    [8, 9]  
4             0.0     0.0            0.0  [10, 11, 12, 13, 14, 15]  
